In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ad-displayclick-data-on-taobaocom/user_profile.csv
/kaggle/input/ad-displayclick-data-on-taobaocom/raw_sample.csv
/kaggle/input/ad-displayclick-data-on-taobaocom/ad_feature.csv


In [18]:
user_profile = pd.read_csv("/kaggle/input/ad-displayclick-data-on-taobaocom/user_profile.csv")
raw_sample = pd.read_csv("/kaggle/input/ad-displayclick-data-on-taobaocom/raw_sample.csv")
ad_feature = pd.read_csv("/kaggle/input/ad-displayclick-data-on-taobaocom/ad_feature.csv")

In [33]:
data = raw_sample.merge(ad_feature, left_on='adgroup_id', right_on="adgroup_id")
data = data.merge(user_profile, left_on='user', right_on="userid")

df = data[['user', 'time_stamp', 'adgroup_id', 'clk', 'cate_id',
         'brand', 'price', 'final_gender_code', 'age_level', 
       'shopping_level', 'occupation' ]]

In [34]:
# remove all brands with less than 1000 occurences
print(f"Number of rows: {len(df)}")
mask = df['brand'].value_counts() >= 1000
indices = mask[mask].index
df = df[df['brand'].isin(indices)]
print(f"Number of rows after deletion of brands: {len(df)}")

Number of rows: 25029435
Number of rows after deletion of brands: 9739145


In [37]:
user_row_counts = df['user'].value_counts()
print(user_row_counts)

402587     487
111091     384
551596     382
570179     324
883667     314
          ... 
822234       1
395413       1
408401       1
1103031      1
500873       1
Name: user, Length: 492046, dtype: int64


In [38]:
users_with_more_than_10_entries = df['user'].value_counts().loc[lambda x: x > 10].index
df = df[df['user'].isin(users_with_more_than_10_entries)]
print(f"Number of rows after deletion of users with less than 10 entries: {len(df)}")

Number of rows after deletion of users with less than 10 entries: 710834


In [39]:
user_row_counts = df['user'].value_counts()
print(user_row_counts)

402587    487
111091    384
551596    382
570179    324
883667    314
         ... 
831563     11
776982     11
855572     11
750339     11
381594     11
Name: user, Length: 29255, dtype: int64


In [40]:
df.dropna(inplace=True)
df['time_stamp'] = pd.to_datetime(df['time_stamp'], unit='s')
df.sort_values(by=['user', 'time_stamp'], inplace=True)
df['time_idx'] = df.groupby('user').cumcount()

In [41]:
filtered_df = df[df['time_idx'] < 10]
len(filtered_df)

292550

In [43]:
user_row_counts = filtered_df['user'].value_counts()
print(user_row_counts)

7          10
760597     10
760548     10
760526     10
760483     10
           ..
383689     10
383675     10
383665     10
383628     10
1141697    10
Name: user, Length: 29255, dtype: int64


In [53]:
filtered_df.to_csv("/kaggle/working/df_updated.csv",index=False)

In [50]:
filtered_df.groupby("brand").value_counts()

brand     user     time_stamp           adgroup_id  clk  cate_id  price  final_gender_code  age_level  shopping_level  occupation  time_idx
46.0      3032     2017-05-06 09:09:17  816838      0    6183     85.0   1                  4          2               0           1           1
                   2017-05-07 01:30:31  504132      0    6183     99.0   1                  4          2               0           2           1
          738246   2017-05-06 08:42:56  816838      0    6183     85.0   1                  3          3               0           1           1
          749442   2017-05-12 06:30:27  748512      0    6183     160.0  2                  5          3               0           8           1
                   2017-05-12 06:28:54  559636      0    6183     194.0  2                  5          3               0           5           1
                                                                                                                                       

In [7]:
df.dropna(inplace=True)
df['time_stamp'] = pd.to_datetime(df['time_stamp'], unit='s')
df.sort_values(by=['user', 'time_stamp'], inplace=True)
df['time_idx'] = df.groupby('user').cumcount()

,user,time_stamp,adgroup_id,clk,cate_id,brand,price,final_gender_code,age_level,shopping_level,occupation,new_user_class_level
9,581738,1494293760,124864,0,6735,247789.0,109.0,1,2,3,0,NaN
15,581738,1494490283,162527,0,10536,370203.0,2680.0,1,2,3,0,NaN
21,581738,1494076831,220169,0,9687,417898.0,8970.0,1,2,3,0,NaN
25,581738,1494491581,264510,0,9498,366492.0,1396.0,1,2,3,0,NaN
28,581738,1494137644,316961,0,6438,452457.0,99.0,1,2,3,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
25029408,456248,1494667283,842405,0,6261,14133.0,119.9,2,4,3,0,2.0
25029409,460743,1494613921,842712,0,6246,92159.0,129.0,2,2,2,0,2.0
25029414,1052098,1494599095,844273,1,6300,97720.0,188.0,2,4,3,0,NaN
25029417,500873,1494599605,844803,0,4505,98931.0,309.0,1,3,1,0,NaN
